<a href="https://colab.research.google.com/github/iMayK/CRUSH_demo/blob/main/CRUSH_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/iMayK/CRUSH_demo.git

In [ ]:
cd CRUSH_demo/demo_FS_basic/

In [ ]:
# download document indexes
!gdown --output utils/ 1iT92VJj-HWlxa9hj3BA8YAkPWGaKLvl3

In [ ]:
!pip install openai

In [ ]:
from getpass import getpass

from utils.utils import ndap_pipeline

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
api_key = getpass('AZURE_OPENAI_API_KEY: ')
endpoint = getpass('AZURE_OPENAI_ENDPOINT: ')

AZURE_OPENAI_API_KEY: ··········
AZURE_OPENAI_ENDPOINT: ··········


In [ ]:
correct_txt_sql_pairs = []

In [ ]:
question = input('\nQuestion: ')
ndap_pipeline(question, api_key, endpoint, correct_txt_sql_pairs)


question: what is the correlation between child nourishment and parental education?

Generated segments:  ['family_health_survey.child_age', 'family_health_survey.child_nourishment', 'population_census.age', 'population_census.male_literate_population', 'population_census.female_literate_population']

Input prompt:
 Please write SQL for "what is the correlation between child nourishment and parental education?" based on the following schema:

 "Primary Population Census 2001 7097" (
	"Male illiterate population,"
	"Male literate population,"
	"Female illiterate population,"
	"Female literate population,"
	"Country,"
	"State LGD Code,"
	"State,"
	"District LGD Code,"
	"District,"
	"Sub-District LGD Code,"
	"Sub-District,"
	"ULB_RLB_Village LGD Code,"
	"ULB_RLB_Village,"
	"YearCode,"
	"Year,"
	"Residence type,"
	"Location Level,"
),
"National Family Health Survey - 4 : District 7034" (
	"Children age group 6 to 23 months receiving an adequate diet (%),"
	"Non-breastfeeding children age 

In [ ]:
check = input('Generated SQL maybe incorrect. Want to register correct SQL? Press "y" to `proceed` and "n" to `skip`: ')
if check == 'y':
    correct_sql = input('Enter correct SQL: ')
    correct_txt_sql_pairs.append(
        {
            'question': question,
            'sql': correct_sql,
        }
    )

Generated SQL maybe incorrect. Want to register correct SQL? Press "y" to `proceed` and "n" to `skip`: n


In [9]:
feedback_options = [
    'Retrieval is wrong',
    'Retrieval is fine, but some other relevant table/columns are missing',
    'Retrieval is correct, but generated SQL is wrong',
    'Generated SQL seems correct, execution error',
    'Everything looks fine!'
]

columns = [
    'question',
    'isWrongRetrieval',
    'wrong_retrieval_feedback',
    'isIncompleteRetrieval',
    'additional_table_feedback',
    'isSqlWrongUnsure',
    'additional_SQL_feedback',
    'isNonExecutable',
    'final_remarks'
]


In [7]:
def get_feedback(question, predicted_sql):
    new_row = [question]
    print('Feedback options:')
    for idx, option in enumerate(feedback_options):
        print(f'{idx+1}: {option}')
    choosen_option = int(input(f'\nYour input: (press any number from 1 .. {len(feedback_options)}): '))

    if choosen_option == 1:
        response = input('\nCan you provide correct table(s)? Press `y` for yes, `n` for no: ')
        correct_tables = "-"
        if response == 'y':
            correct_tables = input('\nPlease enter correct table names/codes:\n')
        new_row.append(["yes", correct_tables, "NA", 'NA', "NA", 'NA', "NA", 'NA'])

    elif choosen_option == 2:
        new_row.extend(["no", '-']) # skipping 1st option details

        response = input('\nCan you provide additional correct table(s)? Press `y` for yes, `n` for no: ')
        additional_tables = "-"
        if response == 'y':
            additional_tables = input('\nPlease enter additional relevant table names/codes (1 in each line):\n')
        new_row.extend(["yes", additional_tables])

        response = int(input('\nIs the SQL correct based on the retrieval? (press `1` for yes, `2` for no, `3` for not sure): '))
        correct_sql = "NA"
        if response in [2, 3]:
            response = input('\nWant to register correct SQL? Press `y` for yes, `n` for no: ')
            if response == 'y':
                correct_sql = input('\nPlease enter correct SQL:\n')
            new_row.extend(["yes", correct_sql])
        else:
            new_row.extend(["no", predicted_sql])

        new_row.extend(["NA", "NA"])

    elif choosen_option == 3:
        new_row.extend([0, "-", 0, "-"]) # skipping initial 2 options

        response = input('\nWant to register correct SQL? Press `y` for yes, `n` for no: ')
        correct_sql = "NA"
        if response == 'y':
            correct_sql = input('\nPlease enter correct SQL:\n')
        new_row.extend(["yes", correct_sql])

        new_row.extend(["yes", "NA"])

    elif choosen_option == 4:
        new_row.extend(["no", "NA", "no", "NA", "no", predicted_sql, "yes", "NA"])
        print('\nThank you for the feedback!')

    elif choosen_option == 5:
        new_row.extend(["no", "NA", "no", "NA", "no", predicted_sql, "no", "all good"])
        print('\nThank you for the feedback!')

def submit_response(new_row):
    worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1PTiGJcXDntJNPVjkFRdgSersW4HqkICQtDc7zezr6w8/edit#gid=0').sheet1

    # get_all_values gives a list of rows.
    rows = worksheet.get_all_values()

    # Find the last row with data
    last_row = len(rows)

    # Append the feedbacks to the first column
    for i, feedback in enumerate(new_row):
        worksheet.update_cell(last_row + 1, i + 1, feedback)


In [8]:
response = input('Want to provide feedback? Press `y` for yes, `n` for no: ')
if response == 'y':
    feedback_data = get_feedback()
    submit_response(feedback_data)

Want to provide feedback? Press `y` for yes, `n` for no: n
